# Dynamics Example
This notebook contains multiple examples of orbit propagations

- Two body analytical 
- Two body numerical (propagating in Cartesian)
- N-body numerical

In [4]:
import pylupnt as pnt
import numpy as np

ModuleNotFoundError: No module named 'pylupnt'

## Step1: Set the initial orbit elements of the orbit

### 1-1. Setup the Orbit Elements 
The first step is to setup the states of the Orbital Elements
For Keplarian orbit elements (classical, equinoctial, QuasiNonsing), you can propagate the orbit by directly updating the state class in the propagator

In [ ]:
# ELFO orbital elements
a = 6541.4
e = 0.60
inc = np.deg2rad(65.5)
Omega = np.deg2rad(0.0)
w = np.deg2rad(90.0)
M = np.deg2rad(0.0)

coe_array = np.array([a, e, inc, Omega, w, M])

# Orbit Elements (Classical)
class_oe = pnt.ClassicalOE(coe_array, frame=pnt.Frame.MOON_CI)  # classical orbital elements class (the second argument is the frame of reference)
x_oe = class_oe.vector
print(" ")
print("Classical orbital elements:")
print(x_oe)

# Orbital Elements (Quasi-Nonsing)
qn_array = pnt.classical_to_quasi_nonsingular(coe_array, mu=pnt.GM_MOON)
class_qn = pnt.QuasiNonsingOE(qn_array, frame=pnt.Frame.MOON_CI)  # quasi-nonsingular orbital elements class
x_qn = class_qn.vector
print(" ")
print("Quasi-nonsingular orbital elements:")
print(x_qn)

### 1-2  Get the Cartesian States

In [ ]:
class_cart = pnt.classical2cartesian(class_oe, mu=pnt.GM_MOON)
x_cart = class_cart.vector

# or you can directly convert the classical orbital elements to cartesian state vector
x_cart2 = pnt.classical2cartesian(coe_array, mu=pnt.GM_MOON)

print("Cartesian State Class", class_cart)
print("Cartesian State Vec (Class->Class)  :", x_cart)
print("Cartesian State Vec (Vec->Vec):", x_cart2)

## Step2: Propagate the States Using Dynamics
We propagate the orbit using three different types of dynamics types

### Case 2-1:  Keplarian Dynamics
This dynamics class propagates the orbit elements in a two-setting. No perturbations are added.
If the dynamics is keplarian, it is more accurate and computationally efficient then numerically propagating the states in Cartesian

In [ ]:
# dynamics
mu_moon = pnt.GM_MOON
keplerian_dynamics = pnt.KeplerianDynamics(mu_moon)

For keplarian dynamics, there is no fuction to directly output the array of state vectors. Therefore, we need to run a for-loop and extract the state vectors at each timestep. 

In [ ]:
T_orbit = 2 * np.pi * np.sqrt(pow(a, 3) / mu_moon)
print(" Orbital period [hrs]: {0:.2f} ".format(T_orbit/3600))

# Propagate the orbit for full orbit
t_end = 0.5 * T_orbit
t_array = np.arange(0, t_end, 60.0)  # time array (60 second steps)

# propagate the classical orbital elements
coe_store = np.zeros((len(t_array), 6))
qn_store = np.zeros((len(t_array), 6))

coe_store[0, :] = class_oe.vector
qn_store[0, :] = class_qn.vector

for i, t in enumerate(t_array[1:]):
    tidx = i + 1
    dt = t_array[tidx] - t_array[tidx - 1]
    keplerian_dynamics.propagate(class_oe, dt)   # propagate the classical orbital elements
    keplerian_dynamics.propagate(class_qn, dt)   # propagate the quasi-nonsingular orbital elements

    coe_store[tidx, :] = class_oe.vector
    qn_store[tidx, :] = class_qn.vector

print(" ")
print(" Final Classical Orbital Elements: ", class_oe)  
print(" ")
print(" Final Quasi-Nonsing Orbital Elements: ", class_qn)


### Visualization

In [ ]:
# Plot the results
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 3, figsize=(8, 6))
oe_labels = ['a', 'e', 'inc', 'Omega', 'w', 'M']
for i in range(6):
    ax[i//3, i%3].plot(t_array, coe_store[:, i], label=oe_labels[i])
    ax[i//3, i%3].set_xlabel('Time [s]')
    ax[i//3, i%3].set_ylabel('Value')
    ax[i//3, i%3].set_title('Element ' + str(i))
    ax[i//3, i%3].legend()
    ax[i//3, i%3].grid()

plt.suptitle('Propagated Classical Orbital Elements in Two Body')
plt.tight_layout()
plt.show()


fig, ax = plt.subplots(2, 3, figsize=(8, 6))
# quasi-nonsingular orbital elements
qn_labels = ['a','u', 'ex', 'ey', 'i', 'Omega']

for i in range(6):
    ax[i//3, i%3].plot(t_array, qn_store[:, i], label=qn_labels[i])
    ax[i//3, i%3].set_xlabel('Time [s]')
    ax[i//3, i%3].set_ylabel('Value')
    ax[i//3, i%3].set_title('Element ' + str(i))
    ax[i//3, i%3].legend()
    ax[i//3, i%3].grid()

plt.suptitle('Propagated Quasi-nonsingular Elements in Two Body')
plt.tight_layout()
plt.show()


### Case 2-2: Cartesian Two-Body Dynamics

In [ ]:
T_orbit = 2 * np.pi * np.sqrt(pow(a, 3) / mu_moon)
t_end = 1 * T_orbit

# define Cartesian Two-Body Dynamics
cart_tb_dyn = pnt.CartesianTwoBodyDynamics(mu_moon)


It is possible to directly get the vector outputs

In [ ]:
# Propagate the orbit for full orbit
t_array_tb = np.arange(0, t_end, 60.0)  # time array (60 second steps)
x_store_tb = np.zeros((len(t_array_tb), 6))   

x_store_tb[0, :] = x_cart
for i, t in enumerate(t_array_tb[1:]):
    tidx = i + 1
    dt = t_array_tb[tidx] - t_array_tb[tidx - 1]
    x_prop = cart_tb_dyn.propagate(state=x_store_tb[tidx-1, :], t0=t_array_tb[tidx-1], tf=t_array_tb[tidx], dt=dt)
    x_store_tb[tidx, :] = x_prop

print(" ")
print("Final Cartesian State Vec: ", x_prop)


###  3D Trajectory Visualization in Matplotlib

In [ ]:
plot3d = pnt.plot.Plot3D(azim=-60, elev=10, figsize=(8,8))
fig = plot3d.fig
ax = plot3d.ax
plot3d.plot_surface(pnt.MOON)
plot3d.plot(x_store_tb)
ax.set_title('Propagated Cartesian State Vec in Two Body')


### 3D Visualization in Plotly
You can also generate a 3d visualizaiton in Plotly, where you can enjoy interactive rotation and zooming

In [ ]:
figp = pnt.plot.plot_constellation(rv=x_store_tb, 
                                   scene=dict(
        xaxis_tickvals=np.arange(-10, 10, 2) * 1e3,
        yaxis_tickvals=np.arange(-10, 10, 2) * 1e3,
        zaxis_tickvals=np.arange(-10, 10, 2) * 1e3,
    ))
figp.add_trace(
    pnt.plot.get_moon_trace()
)
figp.show()

### Case 3-3: N-Body Dynamics Propagation
Next, we will propagate the orbit in N-Body Dynamics. First, we set up the propagator class by registering the bodies which its gravity will be considered

For this example, we will add
- Moon with 2x2 spherical harmonics
- Earth with no spherical harmonics

In [ ]:
# Define the orbit propagation time and time step
t_array_nb = t_array_tb 

# Define the dynamics class
cart_nb_dyn = pnt.NBodyDynamics()
cart_nb_dyn.set_primary_body(pnt.Body.Moon(2, 2))
cart_nb_dyn.add_body(pnt.Body.Earth())

Next, we will propagate the orbit.

Note that in the N-body example, the location of the planet affects the dynamics model. 

Therefore, when passing time to the propagator, we need to pass the TAI time instead of the simulation time.

See the following example to see how we extract the TAI information

In [ ]:
x_store_nb = np.zeros((len(t_array_nb), 6))

tai_t0 = pnt.SpiceInterface.string_to_tai("2022/08/01 01:00:00.000 UTC")
tai_array_nb = tai_t0 + t_array_nb

x_store_nb[0, :] = x_cart
for i, t in enumerate(tai_array_nb[1:]):
    tidx = i + 1
    dt = tai_array_nb[tidx] - tai_array_nb[tidx - 1]
    x_prop = cart_nb_dyn.propagate(state=x_store_nb[tidx-1, :], t0=tai_array_nb[tidx-1], tf=tai_array_nb[tidx], dt=10)
    x_store_nb[tidx, :] = x_prop
    
print("Initial Cartesian State Vec: ", x_cart)
print("Final Cartesian State Vec: ", x_prop)

### 3D Trajectory Visualization in Plotly

In [ ]:
figp = pnt.plot.plot_constellation(rv=x_store_nb, 
                                   scene=dict(
        xaxis_tickvals=np.arange(-10, 10, 2) * 1e3,
        yaxis_tickvals=np.arange(-10, 10, 2) * 1e3,
        zaxis_tickvals=np.arange(-10, 10, 2) * 1e3,
    ))
figp.add_trace(
    pnt.plot.get_moon_trace()
)
figp.show()

We will see the difference in the propagated orbit between the two-body dynamics model and the higher fidelity N-body model

In [ ]:
# compare the two body and n-body dynamics
fig, ax = plt.subplots(3, 2, figsize=(8, 6))
labels = ['x [km]', 'y [km]', 'z [km]', 'vx [km/s]', 'vy [km/s]', 'vz [km/s]']

for i in range(6):
    ax[i//2, i%2].plot(t_array_nb/3600, x_store_nb[:, i], label='N-Body')
    ax[i//2, i%2].plot(t_array_tb/3600, x_store_tb[:, i], label='Two-Body')
    ax[i//2, i%2].set_xlabel('Time [hrs]')
    ax[i//2, i%2].set_ylabel(labels[i])
    ax[i//2, i%2].set_title(labels[i])
    ax[i//2, i%2].legend()
    ax[i//2, i%2].grid()

plt.suptitle('Propagated Cartesian State Vec in Two Body and N-Body')
plt.tight_layout()
plt.show()


# plot the difference
fig, ax = plt.subplots(3, 2, figsize=(8, 6))
labels = ['x [km]', 'y [km]', 'z [km]', 'vx [km/s]', 'vy [km/s]', 'vz [km/s]']

for i in range(6):
    ax[i//2, i%2].plot(t_array_nb/3600, x_store_nb[:, i] - x_store_tb[:, i])
    ax[i//2, i%2].set_xlabel('Time [hrs]')
    ax[i//2, i%2].set_ylabel(labels[i])
    ax[i//2, i%2].set_title(labels[i])
    ax[i//2, i%2].grid()

plt.suptitle('Difference between Two-Body and N-Body Dynamics')
plt.tight_layout()
plt.show()